# Introduction

This is an interactive [Jupyter Book](https://jupyterbook.org/intro.html) containing example problems in thermodynamics, in the format of Jupyter notebooks, developed to supplement the course ME 540, Intermediate Thermodynamics, taught at [Oregon State University](https://oregonstate.edu) by [Prof. Kyle Niemeyer](https://mime.oregonstate.edu/people/kyle-niemeyer).

These example problems are adapted from those given in the (excellent) textbook *Thermodynamics* by Sanford Klein and Gregory Nellis (2017, Cambridge University Press, 3rd edition). The original examples and problems in this book were solved using EES (Engineering Equation Solver), while the examples here use Python combined with [Cantera](https://cantera.org) and the [SciPy](https://www.scipy.org) ecosystem of packages, along with [Pint](https://pint.readthedocs.io) for converting units.
All artwork here is original.


## Solving thermodynamics problems

This module introduces how to solve thermodynamics problems in Python using Cantera and Pint. It will also briefly show how [CoolProp](http://coolprop.org) could also be used, if you need access to a wider range of fluids than Cantera currently supports.

First, we need to import the necessary libraries:

In [1]:
# Numpy adds some useful numerical types and functions
import numpy as np

# Cantera will handle thermodynamic properties
import cantera as ct

# Pint gives us some helpful unit conversion
from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity # We will use this to construct quantities (value + unit)

## Quick Pint intro



In [81]:
# Water
f = ct.Water()

# specify temperature and specific volume, in SI units
f.TV = 673.15, 1e-2
f()


  water:

       temperature          673.15  K
          pressure     1.99362e+07  Pa
           density             100  kg/m^3
  mean mol. weight          18.016  amu
    vapor fraction               1

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy    -1.31504e+07       -2.369e+08     J
   internal energy    -1.33497e+07       -2.405e+08     J
           entropy         9078.37        1.636e+05     J/K
    Gibbs function    -1.92615e+07        -3.47e+08     J
 heat capacity c_p         6284.65        1.132e+05     J/K
 heat capacity c_v         2693.99        4.853e+04     J/K



Using Cantera to get fluid properties, for R134a:

In [40]:
f = ct.Hfc134a()

# Specify temperature and pressure
temp = Q_(325, 'K')
pres = Q_(250e3, 'Pa')

f.TP = temp.to('K').magnitude, pres.to('Pa').magnitude
# see overview of properties
f()


  hfc134a:

       temperature             325  K
          pressure          250000  Pa
           density         9.80674  kg/m^3
  mean mol. weight         102.032  amu
    vapor fraction               1

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          245639        2.506e+07     J
   internal energy          220146        2.246e+07     J
           entropy         1629.17        1.662e+05     J/K
    Gibbs function         -283840       -2.896e+07     J
 heat capacity c_p         912.086        9.306e+04     J/K
 heat capacity c_v         812.939        8.295e+04     J/K



In [19]:
# get internal energy in SI units (mass basis)
print(f'Internal energy: {f.u: .2f} J/kg')

Internal energy:  220146.17 J/kg


In [41]:
# specify pressure and internal enegy
pres = Q_(250e3, 'Pa')
internal_energy = Q_(300e3, 'J/kg')
f.UP = internal_energy.to('J/kg').magnitude, pres.to('Pa').magnitude

# get specific volume
print(f'Specific volume: {f.v: 0.4f} m^3/kg')

Specific volume:  0.1332 m^3/kg


## Compare ideal and real gas behavior for air

In [85]:
temp1 = Q_(500, 'K')
temp2 = Q_(1270, 'K')
pres = Q_(500, 'kPa')

# Real gas
air_real1 = ct.Nitrogen()
air_real1.TP = temp1.to('K').magnitude, pres.to('Pa').magnitude 

air_real2 = ct.Nitrogen()
air_real2.TP =  temp2.to('K').magnitude, pres.to('Pa').magnitude 

delta_u_real = Q_(air_real2.u - air_real1.u, 'J/kg')
print(f'Δu for real gas: {delta_u_real: .2f}')

# ideal gas
air_ideal1 = ct.Solution('air.cti')
air_ideal1.TPX = temp1.to('K').magnitude, pres.to('Pa').magnitude, 'N2:1.0'

air_ideal2 = ct.Solution('air.cti')
air_ideal2.TPX = temp2.to('K').magnitude, pres.to('Pa').magnitude, 'N2:1.0'

delta_u_ideal = Q_(air_ideal2.u - air_ideal1.u, 'J/kg')
print(f'Δu for ideal gas: {delta_u_ideal: .2f}')

diff = 100*np.abs(delta_u_ideal-delta_u_real)/delta_u_real
print(f'difference: {diff.magnitude: .2f}%')

Δu for real gas: 647294.70 joule / kilogram
Δu for ideal gas: 648472.39 joule / kilogram
difference:  0.18%
